In [ ]:
#r "nuget:Microsoft.DotNet.Interactive.AspNetCore, 1.0.0-beta.23562.1"

In [ ]:
#r "nuget:Microsoft.DotNet.Interactive.AI, 1.0.0-beta.23567.4"


In [ ]:
#!value --name key
YOUR AZURE OPEN AI KEY

In [ ]:
#!value --name endpoint
https://your-enpoint.openai.azure.com/

In [ ]:
#!connect azure-openai --model-type TextEmbeddingGenerator --kernel-name knowledge --api-key @value:key --endpoint @value:endpoint --deployment text-embedding-ada-002

In [ ]:
#!connect azure-openai --model-type ChatCompletion --kernel-name chat --api-key @value:key --endpoint @value:endpoint --deployment gpt-35-turbo-16k --use-knowledge knowledge

In [ ]:
#!function bullet_points --skill writer 
Generate bullet points take aways from the following text:
{{$input}}

In [ ]:
#!use-skills function.writer.bullet_points
Napoleon Bonaparte (born Napoleone Buonaparte;[a] 15 August 1769 – 5 May 1821), later known by his regnal name Napoleon I, was a Corsican-born French military commander and political leader who rose to prominence during the French Revolution and led successful campaigns during the Revolutionary Wars. He was the de facto leader of the French Republic as First Consul from 1799 to 1804, then Emperor of the French from 1804 until 1814 and again in 1815. Napoleon's political and cultural legacy endures to this day, as a highly celebrated and controversial leader. He initiated many liberal reforms that have persisted in society, and is considered one of the greatest military commanders in history. His campaigns are still studied at military academies worldwide. Between three and six million civilians and soldiers died in what became known as the Napoleonic Wars.[2][3]

Napoleon was born on the island of Corsica, not long after its annexation by France, to a native family descending from minor Italian nobility.[4][5] He supported the French Revolution in 1789 while serving in the French army, and tried to spread its ideals to his native Corsica. He rose rapidly in the Army after he saved the governing French Directory by firing on royalist insurgents. In 1796, he began a military campaign against the Austrians and their Italian allies, scoring decisive victories and becoming a national hero. Two years later, he led a military expedition to Egypt that served as a springboard to political power. He engineered a coup in November 1799 and became First Consul of the Republic.

Differences with the United Kingdom meant France faced the War of the Third Coalition by 1805. Napoleon shattered this coalition with victories in the Ulm campaign, and at the Battle of Austerlitz, which led to the dissolution of the Holy Roman Empire. In 1806, the Fourth Coalition took up arms against him. Napoleon defeated Prussia at the battles of Jena and Auerstedt, marched the Grande Armée into Eastern Europe, and defeated the Russians in June 1807 at Friedland, forcing the defeated nations of the Fourth Coalition to accept the Treaties of Tilsit. Two years later, the Austrians challenged the French again during the War of the Fifth Coalition, but Napoleon solidified his grip over Europe after triumphing at the Battle of Wagram.

Hoping to extend the Continental System, his embargo against Britain, Napoleon invaded the Iberian Peninsula and declared his brother Joseph the King of Spain in 1808. The Spanish and the Portuguese revolted in the Peninsular War aided by a British army, culminating in defeat for Napoleon's marshals. Napoleon launched an invasion of Russia in the summer of 1812. The resulting campaign witnessed the catastrophic retreat of Napoleon's Grande Armée. In 1813, Prussia and Austria joined Russian forces in a Sixth Coalition against France, resulting in a large coalition army defeating Napoleon at the Battle of Leipzig. The coalition invaded France and captured Paris, forcing Napoleon to abdicate in April 1814. He was exiled to the island of Elba, between Corsica and Italy. In France, the Bourbons were restored to power.

Napoleon escaped in February 1815 and took control of France.[6] The Allies responded by forming a Seventh Coalition, which defeated Napoleon at the Battle of Waterloo in June 1815. The British exiled him to the remote island of Saint Helena in the Atlantic, where he died in 1821 at the age of 51.

Napoleon had an extensive impact on the modern world, bringing liberal reforms to the lands he conquered, especially the regions of the Low Countries, Switzerland and parts of modern Italy and Germany. He implemented many liberal policies in France and Western Europe.[b]

In [ ]:
#!function answer --skill writer 
answer the following question in a few sentences:
{{$input}}

In [ ]:
#!use-skills function.writer.answer
What is the divine comedy about?

In [ ]:
#!use-skills function.writer.answer function.writer.bullet_points
What is the divine comedy about?

In [ ]:
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Commands;
using Microsoft.DotNet.Interactive.Events;
using Microsoft.DotNet.Interactive.AI;

var kernel = Kernel.Root.FindKernels(k => k is TextCompletionKernel).Single(k => k.Name.StartsWith("chat(text)"));

public async Task<string> AskQuestion(string question){
    var result = await kernel.SendAsync(new SubmitCode($"""
    #!use-skills function.writer.answer function.writer.bullet_points
    {question}
"""));
    var response = result.Events.OfType<ReturnValueProduced>().Last();
    var text = response.FormattedValues.Single(fv => fv.MimeType == "text/plain").Value;
    return text;
}

In [ ]:
 var answer = await AskQuestion("Where was Napoleon born?");


In [ ]:
#!aspnet
using System.IO;
using System.Text.Json;
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Commands;
using Microsoft.DotNet.Interactive.Events;
using Microsoft.DotNet.Interactive.AI;

public record Question(string question);

Endpoints.MapPost("/askquestion", async context =>
{
    var data = await new StreamReader(context.Request.Body).ReadToEndAsync();
    try{
    var question = System.Text.Json.JsonSerializer.Deserialize<Question>(data);
    var response = await AskQuestion(question.question);
    await context.Response.WriteAsync(response);
    }catch(Exception ex){
        await context.Response.WriteAsync(ex.ToString());
    }  
});

In [ ]:
#!aspnet-stop

In [ ]:
var uri = HttpClient.BaseAddress;

In [ ]:
#!set --value @csharp:uri --name uri

In [ ]:
<div id="target_for_question">replace me</div>

In [ ]:

fetch(`${uri}/askquestion`, {
    method: "post",
    headers: {
      'Accept': 'text/plain',
      'Content-Type': 'application/json',
      'Access-Control-Allow-Origin': '*'
    },
    body: JSON.stringify({
      question: 'what did napoleon bonaparte do in military life?'
    })
  })
    .then(response => response.text())
    .then(text => document.getElementById("target_for_question").innerHTML = text);


In [ ]:
<div>
    <input id="question" type="text"/>
    <button id="ask">Ask</button>
    <div id="answer_here">replace me</div>
</div>
<script type="text/javascript">
    document.getElementById("ask").onclick = function() {
        let question = document.getElementById("question").value;
        document.getElementById("ask").enabled = false;
        fetch(`${uri}/askquestion`, {
            method: "post",
            headers: {
              'Accept': 'text/plain',
              'Content-Type': 'application/json',
              'Access-Control-Allow-Origin': '*'
            },
            body: JSON.stringify({
              question: question
            })
          })
        .then(response => response.text())
        .then(text => document.getElementById("answer_here").textContent = text)
        .finally(() => document.getElementById("ask").enabled = true);
    };
</script>